In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.txt', sep="\t")


In [32]:
df = pd.read_csv('data.txt', sep="\t")

In [3]:
pd.set_option('max_columns', None)
data.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip5,nppes_provider_zip4,nppes_provider_state,nppes_provider_country,specialty_description,description_flag,medicare_prvdr_enroll_status,total_claim_count,total_30_day_fill_count,total_drug_cost,total_day_supply,bene_count,ge65_suppress_flag,total_claim_count_ge65,total_30_day_fill_count_ge65,total_drug_cost_ge65,total_day_supply_ge65,bene_count_ge65_suppress_flag,bene_count_ge65,brand_suppress_flag,brand_claim_count,brand_drug_cost,generic_suppress_flag,generic_claim_count,generic_drug_cost,other_suppress_flag,other_claim_count,other_drug_cost,mapd_suppress_flag,mapd_claim_count,mapd_drug_cost,pdp_suppress_flag,pdp_claim_count,pdp_drug_cost,lis_suppress_flag,lis_claim_count,lis_drug_cost,nonlis_suppress_flag,nonlis_claim_count,nonlis_drug_cost,opioid_claim_count,opioid_drug_cost,opioid_day_supply,opioid_bene_count,opioid_prescriber_rate,la_opioid_claim_count,la_opioid_drug_cost,la_opioid_day_supply,la_opioid_bene_count,la_opioid_prescriber_rate,antibiotic_claim_count,antibiotic_drug_cost,antibiotic_bene_count,antipsych_ge65_suppress_flag,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_bene_ge65_suppress_flg,antipsych_bene_count_ge65,average_age_of_beneficiaries,beneficiary_age_less_65_count,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_female_count,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,21502.0,1854.0,MD,US,Internal Medicine,S,E,677,695.5,32639.57,14788,234.0,NaN,516.0,532.533333,23174.78,11428.0,NaN,182.0,#,NaN,NaN,NaN,552.0,5941.53,*,NaN,NaN,NaN,150.0,3032.66,NaN,527.0,29606.91,NaN,305.0,17267.72,NaN,372.0,15371.85,25.0,197.47,134.0,24.0,3.69,NaN,NaN,NaN,NaN,NaN,125.0,2478.65,104.0,*,NaN,NaN,*,NaN,72.0,52.0,82.0,67.0,33.0,142.0,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,43623.0,3536.0,OH,US,Anesthesiology,S,E,1946,2054.8,140189.01,58605,276.0,NaN,881.0,919.066667,50838.50,26089.0,NaN,160.0,NaN,235.0,106048.28,NaN,1711.0,34140.73,NaN,0.0,0.0,NaN,930.0,67081.22,NaN,1016.0,73107.79,NaN,1184.0,94361.07,NaN,762.0,45827.94,1040.0,50895.21,29238.0,200.0,53.44,200.0,37637.86,5853.0,49.0,19.23,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,65.0,116.0,99.0,45.0,16.0,184.0,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,89403.0,7482.0,NV,US,Dentist,S,N,55,55.0,302.01,430,33.0,#,NaN,NaN,NaN,NaN,#,NaN,NaN,0.0,0.00,#,NaN,NaN,*,NaN,NaN,NaN,16.0,58.19,NaN,39.0,243.82,*,NaN,NaN,#,NaN,NaN,14.0,61.95,35.0,11.0,25.45,0.0,0.00,0.0,0.0,0.00,28.0,134.92,22.0,NaN,0.0,0.0,NaN,0.0,72.0,NaN,NaN,NaN,NaN,16.0,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,91744.0,4648.0,CA,US,Dentist,S,N,18,18.0,113.81,150,NaN,#,NaN,NaN,NaN,NaN,#,NaN,NaN,0.0,0.00,NaN,18.0,113.81,NaN,0.0,0.0,*,NaN,NaN,*,NaN,NaN,#,NaN,NaN,*,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,12.0,81.54,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,37243.0,1.0,TN,US,Nurse Practitioner,S,N,90,110.0,7561.21,2681,11.0,NaN,65.0,85.000000,3894.08,1931.0,*,NaN,NaN,28.0,6576.28,NaN,62.0,984.93,NaN,0.0,0.0,NaN,22.0,485.04,NaN,68.0,7076.17,NaN,73.0,3947.56,NaN,17.0,3613.65,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,NaN,0.0,0.00,0.0,NaN,0.0,0.0,NaN,0.0,

In [3]:
"""
In 2017, what was the average number of beneficiaries per provider? Due to the suppression of 
data for those with few beneficiaries, we can only include those with more than 10 beneficiaries.
"""

sum = 0
count = 0
data['bene_count'].fillna(0)
for column in data['bene_count']:
    if column > 10:
        sum += column
        count += 1
        

print("Average number of beneficiaries per provider : % s "
        % (sum/count))

Average number of beneficiaries per provider : 158.3494585173676 


In [29]:
"""
For each provider, estimate the length of the average prescription from the total_day_supply and total_claim_count.
What is the median, in days, of the distribution of this value across all providers?
"""

import statistics 
for column in data[['total_day_supply']]:
    columnSeriesObj = data[column] 
    total_day = columnSeriesObj.values    

for column in data[['total_claim_count']]:
    columnSeriesObj = data[column] 
    total_claim = columnSeriesObj.values

average = []
for i in range(len(total_day)):
    total = total_day[i]/total_claim[i]
    average.append(total)

print("Median of the distribution of days across all providers : % s "
        % (statistics.median(average))) 
    
    


Median of the distribution of days across all providers : 29.7125748502994 


In [5]:
"""
Find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions in each state.
Assume that each beneficiary attends only a single provider. What is the difference between the largest and smallest ratios?
"""

avg = data.groupby(['nppes_provider_state']).mean()
for column in avg[['opioid_claim_count']]:
    columnSeriesObj = avg[column] 
    opiod = columnSeriesObj.values    

for column in avg[['antibiotic_claim_count']]:
    columnSeriesObj = avg[column] 
    antibiotic = columnSeriesObj.values
    

ratios = []
for i in range(len(opiod)):
    ratio = opiod[i]/antibiotic[i]
    ratios.append(ratio)


print("The difference between the largest and smallest ratios of opiod and antibiotic: % s "
        % (max(ratios) - min(ratios)))
    

The difference between the largest and smallest ratios of opiod and antibiotic: 1.5773099424780466 


In [30]:
"""
Work out for each Specialty the fraction of drug claims that are for brand-name drugs.
Include only providers for whom the relevant information has not been suppressed, 
and consider only specialties with at least 1000 total claims. What is the standard deviation of these fractions?
"""


above_1000 = data[data['total_claim_count'] >= 1000]
above_1000.groupby(['specialty_description'])
not_suppressed = above_1000[['brand_claim_count', 'total_claim_count']]
not_suppressed2 = not_suppressed[not_suppressed['brand_claim_count']>0]
not_suppressed2


fractions = []
for index,row in not_suppressed2.iterrows():
    fractions.append((row['brand_claim_count']/row['total_claim_count']))


print("The standard deviation of brand-name drugs fractions: % s "
        % statistics.pstdev(fractions))


The standard deviation of brand-name drugs fractions: 0.12171276759305948 


In [35]:
"""
For each provider where the relevant columns are not suppressed, 
work out the fraction of claims for beneficiaries age 65 and older, 
as well as the fraction of claims for beneficiaries with a low-income subsidy. 
    What is the Pearson correlation coefficient between these values?
"""
def above65(row):
    ab65 = row['beneficiary_age_65_74_count'] + row['beneficiary_age_75_84_count'] + row['beneficiary_age_greater_84_count']
    total = row['total_claim_count']
    return (ab65/total)
    
def subsidy(row):
    lis = row['lis_claim_count']
    total = row['total_claim_count']
    return (lis/total)
    
    
df.fillna(0)
df2 = df[df['total_claim_count']>0]
df2['above65'] = df2.apply(lambda row: above65(row), axis=1)
df2['lis'] = df2.apply(lambda row: subsidy(row), axis=1)


df3 = df2[['above65', 'lis']]
pearsoncorr = df3.corr(method='pearson')
pearsoncorr

,above65,lis
above65,1.000000,-0.166026
lis,-0.166026,1.000000
